In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "showcase"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using LinearAlgebra
using Rotations
using GLMakie
using GeometryBasics

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [39]:
@load "../../Data/paper2/measure_paths/jld2s/test/2.jld2" realizations pdgms idgms ifils Es thetas Vs As Cs Xs OLs

11-element Vector{Symbol}:
 :realizations
 :pdgms
 :idgms
 :ifils
 :Es
 :thetas
 :Vs
 :As
 :Cs
 :Xs
 :OLs

In [52]:
function visualize_persistence_analysis_for_ma_simulation(realizations, pdgms, Es, thetas, Vs, As, Cs, Xs, OLs)
    xs = [i for i in 1:length(pdgms)];
    zero_ps = [MorphoMol.Energies.get_total_persistence(dgm[1], 0.01) for dgm in pdgms]
    one_ps = [MorphoMol.Energies.get_total_persistence(dgm[2], -0.1) for dgm in pdgms]
    two_ps = [MorphoMol.Energies.get_total_persistence(dgm[3], -0.1) for dgm in pdgms]
    tps = [MorphoMol.Energies.get_total_persistence_summed(dgm, [0.1, -0.1, -0.1]) for dgm in pdgms];

    f = Figure(fontsize = 12)
    cm = :seaborn_muted
    cr = (1, 11)
    ms = 5
    title_fs = 15
    #Slider
    sl_i = Slider(f[5, 1:5], range = 1:length(pdgms), startvalue = 1)
    x = sl_i.value

    # Persistence
    cm = :Spectral_4
    cr = (1, 4)

    pg = GridLayout(f[1:2, 3:5])
    Label(pg[0, 1:3], text = "Persistence (not included in Energy)", fontsize = title_fs)
    zero_p_ax = Axis(pg[2, 1], title = L"0.01 \text{total}(b_0)")
    one_p_ax = Axis(pg[1, 2], title = L"-0.1 \text{total}(b_1)")
    two_p_ax = Axis(pg[2, 2], title = L"-0.1 \text{total}(b_2)")
    tp_ax = Axis(pg[1, 1], title = L"Σ")

    tp_mark = @lift(Point2f($x, $@lift(tps[$x])))

    zero_p_mark = @lift(Point2f($x, $@lift(zero_ps[$x])))
    one_p_mark = @lift(Point2f($x, $@lift(one_ps[$x])))
    two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))

    scatter!(zero_p_ax, xs, zero_ps, color = 1, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(zero_p_ax, zero_p_mark, color=:black, markersize = 10)

    scatter!(one_p_ax, xs, one_ps, color = 2, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(one_p_ax, one_p_mark, color=:black, markersize = 10)

    scatter!(two_p_ax, xs, two_ps, color = 3, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(two_p_ax, two_p_mark, color=:black, markersize = 10)

    scatter!(tp_ax, xs, tps, color = 4, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

    pd_ax_a = Axis(pg[1, 3], title = L"b_1", xticks = 0:25:100, yticks = 0:25:100)
    pd_ax_b = Axis(pg[2, 3], title = L"b_2", xticks = 0:25:100, yticks = 0:25:100)
    dgm_points_1 = @lift([Point2f(pdgms[$x][2][i,1], pdgms[$x][2][i,2]) for i in 1:size(pdgms[$x][2])[1]])
    dgm_points_2 = @lift([Point2f(pdgms[$x][3][i,1], pdgms[$x][3][i,2]) for i in 1:size(pdgms[$x][3])[1]])


    scatter!(pd_ax_a, dgm_points_1, color = 2, colormap = cm, colorrange = cr, markersize = ms)
    lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
    scatter!(pd_ax_b, dgm_points_2, color = 3, colormap = cm, colorrange = cr, markersize = ms)
    lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

    #Energy and Theta
    eg = GridLayout(f[3, 1:2])
    Label(eg[0, 1:2], text = "Energy", fontsize = title_fs)
    theta_ax = Axis(eg[1, 1], title = L"\Theta")
    E_ax = Axis(eg[1, 2], title = L"F_{sol}")

    theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
    E_mark = @lift(Point2f($x, $@lift(Es[$x])))

    scatter!(theta_ax, xs, thetas, color = :magenta, markersize = ms)
    scatter!(theta_ax, theta_mark, color= :black, markersize = 10)

    scatter!(E_ax, xs, Es, color = :blue,  markersize = ms)
    scatter!(E_ax, E_mark, color=:black, markersize = 10)
    
    # Configuration
    cg = GridLayout(f[1:2, 1:2])
    Label(cg[0, 1:2], text = "Configuration", fontsize = title_fs)
    conf_ax = LScene(cg[1:2, 1:2])
    n_atoms = size(realizations[1])[2]
    n_mol = 2
    points = @lift([Point3f(realizations[$x][1,i], realizations[$x][2,i], realizations[$x][3,i]) for i in 1:n_atoms])
    colors = vcat([[j for _ in 1:1206] for j in 1:n_mol]...)
    conf_ms = 10

    scatter!(conf_ax, points, markersize = conf_ms, color = colors, colormap = :rainbow)

    #Measures    
    pf = MorphoMol.Energies.get_prefactors(1.4, 0.3665)
    Vs = pf[1] .* Vs
    As = pf[2] .* As
    Cs = pf[3] .* Cs
    Xs = pf[4] .* Xs

    mg = GridLayout(f[4, 1:5])
    Label(mg[0, 1:5], text = "Geometric Measures", fontsize = title_fs)
    cm = :Set2_5
    cr = (1, 5)

    Vs_ax = Axis(mg[1, 1], title = L"pV")
    As_ax = Axis(mg[1, 2], title = L"\sigma A")
    Cs_ax = Axis(mg[1, 3], title = L"\kappa C")
    Xs_ax = Axis(mg[1, 4], title = L"\overline{\kappa} X")
    OLs_ax = Axis(mg[1, 5], title = L"OLs")

    Vs_mark = @lift(Point2f($x, $@lift(Vs[$x])))
    As_mark = @lift(Point2f($x, $@lift(As[$x])))
    Cs_mark = @lift(Point2f($x, $@lift(Cs[$x])))
    Xs_mark = @lift(Point2f($x, $@lift(Xs[$x])))
    OLs_mark = @lift(Point2f($x, $@lift(OLs[$x])))

    scatter!(Vs_ax, xs, Vs, color = 1, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(Vs_ax, Vs_mark, color=:black, markersize = 10)

    scatter!(As_ax, xs, As, color = 2, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(As_ax, As_mark, color=:black, markersize = 10)

    scatter!(Cs_ax, xs, Cs, color = 3, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(Cs_ax, Cs_mark, color=:black, markersize = 10)

    scatter!(Xs_ax, xs, Xs, color = 4, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(Xs_ax, Xs_mark, color=:black, markersize = 10)

    scatter!(OLs_ax, xs, OLs, color = 5, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(OLs_ax, OLs_mark, color=:black, markersize = 10)

    f
end

display(visualize_persistence_analysis_for_ma_simulation(realizations, pdgms, Es, thetas, Vs, As, Cs, Xs, OLs))

GLMakie.Screen(...)

In [41]:
function get_multichromatic_edges_of_tetrahedron(verts; split = 1206)
    a,b,c,d = verts
    combinations = [[a,b], [a,c], [a,d], [b,c], [b,d], [c,d]]
    [e for e in combinations if div(e[1], split) != div(e[2], split)]
end

function get_barycenters_of_multichromatic_edges(edges, points)
    barycenters = Vector{Point3f}([])
    for e in edges
        a,b = e[1],e[2]
        bc = Point3f(0.5 * (points[a+1] + points[b+1]))
        push!(barycenters, bc)
    end
    if length(barycenters) == 4
        bc = Point3f(0.25 * (barycenters[1] + barycenters[2] + barycenters[3] + barycenters[4]))
        push!(barycenters, bc)
    end 
    barycenters
end

function get_faces_from_barycenters(barycenters)
    if length(barycenters) == 3
        faces = [TriangleFace([2, 3, 1])]
    elseif length(barycenters) == 5
        faces = [TriangleFace([2, 4, 5]), TriangleFace([3, 1, 5]), TriangleFace([4, 3, 5]), TriangleFace([1, 2, 5])]
    end
    faces
end

function get_mesh_data(ifil, points)
    bcs = Vector{Point3f}([])
    faces =  Vector{TriangleFace{Int}}([])
    colors = Vector{Float32}([])
    total_vertices = 0
    for (vertices, value) in ifil
        if length(vertices) == 4
            mce = get_multichromatic_edges_of_tetrahedron(vertices)
            t_bcs = get_barycenters_of_multichromatic_edges(mce, points)
            t_faces = get_faces_from_barycenters(t_bcs)
            bcs = [bcs; t_bcs]
            faces = [faces; [f .+ total_vertices for f in t_faces]]
            total_vertices += length(t_bcs)
            colors = [colors; [value for _ in 1:length(t_bcs)]]
        end
    end
    (bcs, faces, colors)
end

get_mesh_data (generic function with 1 method)

In [50]:
function visualize_interface_persistence_analysis_for_ma_simulation(realizations, idgms, ifils, Es, thetas, Vs, As, Cs, Xs, OLs)
    max_v = sqrt(maximum([v for ifil in ifils for (_, v) in ifil]))
    min_v = sqrt(minimum([v for ifil in ifils for (_, v) in ifil]))

    meshdata = [get_mesh_data(ifil, [Vector{Float64}(e) for e in eachcol(realization)]) for (ifil, realization) in zip(ifils, realizations)]

    xs = [i for i in 1:length(idgms)];
    # pw = [0.1, 0.1]
    # one_ps = [MorphoMol.Energies.get_total_persistence(dgm[1], pw[1]) for dgm in idgms]
    # two_ps = [MorphoMol.Energies.get_total_persistence(dgm[2], pw[2]) for dgm in idgms]
    # tps = [MorphoMol.Energies.get_total_persistence_summed(dgm, pw) for dgm in idgms];
    pw = [-0.01, -0.01]
    one_ps = [MorphoMol.Energies.get_divided_persistence(dgm[1], pw[1]) for dgm in idgms]
    two_ps = [MorphoMol.Energies.get_divided_persistence(dgm[2], pw[2]) for dgm in idgms]
    tps = [MorphoMol.Energies.get_divided_persistence_summed(dgm, pw) for dgm in idgms];

    f = Figure(fontsize = 12)
    title_fs = 15
    cm = :Dark2_8
    cr = (1,8)

    #Slider
    sl_i = Slider(f[6, 1:6], range = 1:length(idgms), startvalue = 1)
    x = sl_i.value

    # Interface
    ig = GridLayout(f[1:3, 4:6])
    Label(ig[0, 1:2], text = "Interface", fontsize = title_fs)
    i_sc = LScene(ig[1:2, 1:2], show_axis=false)
    vertices = [md[1] for md in meshdata]
    faces = [md[2] for md in meshdata]
    meshes = [GeometryBasics.Mesh(vs, fs) for (vs, fs) in zip(vertices, faces)]
    colors = [md[3] for md in meshdata]

    l_ms, l_cs = @lift(meshes[$x]), @lift(colors[$x])
    mesh!(i_sc, l_ms, color = l_cs, colorrange = (min_v, max_v), colormap = :magma)


    # Configuration
    cg = GridLayout(f[1:2, 1:3])
    Label(cg[0, 1:2], text = "Configuration", fontsize = title_fs)
    conf_sc = LScene(cg[1:2, 1:2], show_axis=false)
    n_atoms = size(realizations[1])[2]
    n_mol = 2
    points = @lift([Point3f(realizations[$x][1,i], realizations[$x][2,i], realizations[$x][3,i]) for i in 1:n_atoms])
    colors = vcat([[j for _ in 1:1206] for j in 1:n_mol]...)
    conf_ms = 10
    scatter!(conf_sc, points, markersize = conf_ms, color = colors, colormap = :pastel)
    colors = [md[3] for md in meshdata]
    mesh!(conf_sc, l_ms, color = l_cs, colorrange = (min_v, max_v), colormap = :magma)
    
    # Persistence
    cm = :Spectral_4
    cr = (1, 4)
    pg = GridLayout(f[3:4, 1:3])
    Label(pg[0, 1:3], text = "Persistence (not included in Energy)", fontsize = title_fs)

    pd_ax_a = Axis(pg[1, 3], title = "codim 1", xticks = 0:25:100, yticks = 0:25:100)
    pd_ax_b = Axis(pg[2, 3], title = "codim 2", xticks = 0:25:100, yticks = 0:25:100)
    dgm_points_2 = @lift([Point2f(idgms[$x][1][i,1], idgms[$x][1][i,2]) for i in 1:size(idgms[$x][1])[1]])
    dgm_points_3 = @lift([Point2f(idgms[$x][2][i,1], idgms[$x][2][i,2]) for i in 1:size(idgms[$x][2])[1]])

    ms = 5
    scatter!(pd_ax_a, dgm_points_2, color = 2, colormap = cm, colorrange = cr, markersize = ms)
    lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
    scatter!(pd_ax_b, dgm_points_3, color = 3, colormap = cm, colorrange = cr, markersize = ms)
    lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

    one_p_ax = Axis(pg[1, 2], title = "$(pw[1]) * total 1 persistence")
    two_p_ax = Axis(pg[2, 2], title = "$(pw[2]) * total 2 persistence")
    tp_ax = Axis(pg[1, 1], title = "Total Persistences Summed")

    tp_mark = @lift(Point2f($x, $@lift(tps[$x])))

    one_p_mark = @lift(Point2f($x, $@lift(one_ps[$x])))
    two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))

    scatter!(one_p_ax, xs, one_ps, color = 2, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(one_p_ax, one_p_mark, color=:black, markersize = 10)

    scatter!(two_p_ax, xs, two_ps, color = 3, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(two_p_ax, two_p_mark, color=:black, markersize = 10)

    scatter!(tp_ax, xs, tps, color = 4, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(tp_ax, tp_mark, color = :black, markersize = 10)


    #Energy and Theta
    eg = GridLayout(f[4, 4:6])
    Label(eg[0, 1:2], text = "Energy", fontsize = title_fs)
    E_ax = Axis(eg[1:2, 1], title = L"F_{sol}")
    theta_ax = Axis(eg[1:2, 2], title = L"\Theta")

    theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
    E_mark = @lift(Point2f($x, $@lift(Es[$x])))

    scatter!(theta_ax, xs, thetas, color = :magenta, markersize = ms)
    scatter!(theta_ax, theta_mark, color=:black, markersize = 10)

    scatter!(E_ax, xs, Es, color = :blue, markersize = ms)
    scatter!(E_ax, E_mark, color=:black, markersize = 10)

    #Measures    
    pf = MorphoMol.Energies.get_prefactors(1.4, 0.3665)
    Vs = pf[1] .* Vs
    As = pf[2] .* As
    Cs = pf[3] .* Cs
    Xs = pf[4] .* Xs

    mg = GridLayout(f[5, 1:5])
    Label(mg[0, 1:5], text = "Geometric Measures", fontsize = title_fs)
    cm = :Set2_5
    cr = (1, 5)

    Vs_ax = Axis(mg[1, 1], title = L"pV")
    As_ax = Axis(mg[1, 2], title = L"\sigma A")
    Cs_ax = Axis(mg[1, 3], title = L"\kappa C")
    Xs_ax = Axis(mg[1, 4], title = L"\overline{\kappa} X")
    OLs_ax = Axis(mg[1, 5], title = L"OLs")

    Vs_mark = @lift(Point2f($x, $@lift(Vs[$x])))
    As_mark = @lift(Point2f($x, $@lift(As[$x])))
    Cs_mark = @lift(Point2f($x, $@lift(Cs[$x])))
    Xs_mark = @lift(Point2f($x, $@lift(Xs[$x])))
    OLs_mark = @lift(Point2f($x, $@lift(OLs[$x])))

    scatter!(Vs_ax, xs, Vs, color = 1, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(Vs_ax, Vs_mark, color=:black, markersize = 10)

    scatter!(As_ax, xs, As, color = 2, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(As_ax, As_mark, color=:black, markersize = 10)

    scatter!(Cs_ax, xs, Cs, color = 3, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(Cs_ax, Cs_mark, color=:black, markersize = 10)

    scatter!(Xs_ax, xs, Xs, color = 4, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(Xs_ax, Xs_mark, color=:black, markersize = 10)

    scatter!(OLs_ax, xs, OLs, color = 5, colormap = cm, colorrange = cr, markersize = ms)
    scatter!(OLs_ax, OLs_mark, color=:black, markersize = 10)

    f
end

display(visualize_interface_persistence_analysis_for_ma_simulation(realizations, idgms, ifils, Es, thetas, Vs, As, Cs, Xs, OLs))

GLMakie.Screen(...)

In [35]:
max_v = sqrt(maximum([v for ifil in ifils for (_, v) in ifil]))
min_v = sqrt(minimum([v for ifil in ifils for (_, v) in ifil]))

meshdata = [get_mesh_data(ifil, [Vector{Float64}(e) for e in eachcol(realization)]) for (ifil, realization) in zip(ifils, realizations)]

f = Figure(fontsize = 12)

#Slider
sl_i = Slider(f[2, 1:2], range = 1:length(idgms), startvalue = 1)
x = sl_i.value

# Interface
ig = GridLayout(f[1, 2])
i_sc = LScene(ig[1, 1], show_axis=false)
vertices = [md[1] for md in meshdata]
fcs = [md[2] for md in meshdata]
meshes = [GeometryBasics.Mesh(vs, fs) for (vs, fs) in zip(vertices, fcs)]
colors = [md[3] for md in meshdata]

l_ms, l_cs = @lift(meshes[$x]), @lift(colors[$x])
mesh!(i_sc, l_ms, color = l_cs, colorrange = (min_v, max_v), colormap = :magma)

# Configuration
cg = GridLayout(f[1, 1])
conf_sc = LScene(cg[1, 1], show_axis=false)
n_atoms = size(realizations[1])[2]
n_mol = 2
points = @lift([Point3f(realizations[$x][1,i], realizations[$x][2,i], realizations[$x][3,i]) for i in 1:n_atoms])
colors = vcat([[j for _ in 1:1206] for j in 1:n_mol]...)
conf_ms = 10
scatter!(conf_sc, points, markersize = conf_ms, color = colors, colormap = :pastel)

#i_sc = LScene(cg[1, 1], show_axis=false)
colors = [md[3] for md in meshdata]
mesh!(conf_sc, l_ms, color = l_cs, colorrange = (min_v, max_v), colormap = :magma)



display(f)

GLMakie.Screen(...)